In [1]:
import CCACSP
from preprocessing import *
from import_data import *
data_dir = 'DataCollection/outputs/'
ssvep_dir = data_dir + 'SSVEP/'
mi_dir = data_dir + 'MI/sess2/'
bci_dir1 = '../Documents/OpenBCI_GUI/Recordings/OpenBCISession_2023-02-28_12-03-49/'
bci_dir2 = '../Documents/OpenBCI_GUI/Recordings/OpenBCISession_2023-03-10_14-09-51/'
col_names = ['Fp1', 'O1', 'O2','C3','C4','P3','P4','Pz']
mi_files = get_files(mi_dir)
#mi_files = [mi_files[i] for i in range(len(mi_files)) if (i+1)%6!=0]
bci1_files = get_files(bci_dir1)
bci2_files = get_files(bci_dir2)

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
for i, fi in enumerate(mi_files):
    print("{}: {}".format(i,fi))

0: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MILF.txt
1: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MIRF.txt
2: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MISw.txt
3: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MITy.txt
4: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_NOMI.txt
5: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_baseline.txt
6: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_metadata.txt
7: DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MILF.txt
8: DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MIRF.txt
9: DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MISw.txt
10: DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MITy.txt
11: DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_NOMI.txt
12: DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_baseline.txt
13: DataCollection/outp

In [4]:
def snip_eeg(eeg_df, Fs, snip_len, total_len):
    npts = total_len * Fs
    num_trials = int(len(list(eeg_df['Fp1']))/npts)
    snip_df = pd.DataFrame()
    for chan in eeg_df.columns:
        chan_list = []
        for i in range(num_trials):
            chan_trial = list(eeg_df[chan])[npts*i:npts*i+(total_len-snip_len)*Fs]
            chan_list += chan_trial
        snip_df[chan] = chan_list
    return snip_df

In [130]:
def get_train_data(mi_file, col_names, bci_file, snip_len=1.5):
    eeg, Fs = process_eeg(mi_file, col_names, bci_file, low=2, high=100,snip_len=snip_len)
    eeg_np = eeg.to_numpy()
    npts = int((5-snip_len)*Fs) #5 if no snip
    num_trials = int(eeg_np.shape[0] / npts)
    train_data = np.array([eeg_np[i*npts:(i+1)*npts].reshape(8,npts) for i in range(num_trials)])
    return train_data

In [102]:
swim = get_train_data(mi_files[2], col_names, bci2_files[1])

In [103]:
typing = get_train_data(mi_files[3], col_names, bci2_files[1])

In [8]:
numFilt = 8
filers, clf = CCACSP.train(swim, typing, numFilt)

In [9]:
swim_test = get_train_data(mi_files[8], col_names, bci2_files[1])
typing_test = get_train_data(mi_files[9], col_names, bci2_files[1])

In [10]:
swim_test.shape, typing_test.shape

((4, 8, 750), (4, 8, 750))

In [11]:
output = CCACSP.test(swim_test[2], filers, clf)
output

array([0.])

In [34]:
class_dict = {'lf':0,'rf':1,'swim':2,'type':3,'no':4}

In [36]:
def compare_classes(class1,class2,bci_file):
    c1 = class_dict[class1]
    class1_data = []
    c2 = class_dict[class2]
    class2_data = []
    for sess in range(2):
        class1_file = mi_files[7*sess + c1]
        class1_data.append(get_train_data(class1_file,col_names,bci_file))
        class2_file = mi_files[7*sess + c2]
        class2_data.append(get_train_data(class2_file,col_names,bci_file))
    class1_data = np.concatenate(class1_data)
    class2_data = np.concatenate(class2_data)
    minLen = min(len(class1_data),len(class2_data))
    return class1_data[:minLen],class2_data[:minLen]

In [12]:
def get_session_data(sess_num, bci_file, snip_len):
    data = []
    #for fi in mi_files[5*sess_num:5*(sess_num+1)-1]:
    for fi in mi_files[7*sess_num:7*(sess_num)+4]:
        print(fi)
        data.append(get_train_data(fi, col_names, bci_file))
    class1 = np.concatenate(data)
    class2 = get_train_data(mi_files[7*(sess_num)+4], col_names, bci_file,snip_len)
    return class1, class2

In [35]:
def get_all_data(num_sessions, bci_file, snip_len=1.5):
    mi = []
    nomi = []
    for i in range(num_sessions):
        class1, class2 = get_session_data(i, bci_file, snip_len)
        mi.append(class1)
        nomi.append(class2)
    mi = np.concatenate(mi)
    nomi = np.concatenate(nomi)
    minLen = min(len(mi), len(nomi))
    mi = mi[:minLen]
    nomi = nomi[:minLen]
    return mi, nomi 

In [207]:
right, nomi = compare_classes('rf','no',bci2_files[1])

In [208]:
right.shape, left.shape

((8, 8, 875), (8, 8, 875))

In [147]:
mi, nomi = get_all_data(2, bci2_files[1],snip_len=1)

DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MILF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MIRF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MISw.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MITy.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MILF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MIRF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MISw.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MITy.txt


In [148]:
mi.shape, nomi.shape

((32, 8, 875), (32, 8, 1000))

In [149]:
def get_train_test(mi, nomi, split=0.8):
    np.random.shuffle(mi)
    np.random.shuffle(nomi)
    splitIdx = int(split*len(mi))
    train_mi = mi[:splitIdx]
    test_mi = mi[splitIdx:]
    train_nomi = nomi[:splitIdx]
    test_nomi = nomi[splitIdx:]
    return train_mi, train_nomi, test_mi, test_nomi

In [162]:
def get_results(train_mi, train_nomi, test_mi, test_nomi,show_results=True):
    numFilt = 8
    filters, clf = CCACSP.train(train_mi, train_nomi, numFilt)
    mi_res = []
    nomi_res = []
    for i in range(len(test_mi)):
        mi_result = CCACSP.test(test_mi[i], filters, clf)
        mi_res.append(mi_result[0])
        #print("True class: MI, Predicted: {}".format(mi_result[0]))
        nomi_result = CCACSP.test(test_nomi[i], filters, clf)
        nomi_res.append(nomi_result[0])
        #print("True class: NOMI, Predicted: {}".format(nomi_result[0]))
    decode = {0: 'MI', 1:'NOMI'}
    mi_print = [decode[res] for res in mi_res]
    nomi_print = [decode[res] for res in nomi_res]
    mi_acc = np.sum([1-mi for mi in mi_res]) / len(mi_res)
    nomi_acc = np.sum([nomi for nomi in nomi_res]) / len(nomi_res)
    acc = (mi_acc + nomi_acc) / 2
    #(np.sum([1-mi for mi in mi_res]) + np.sum([nomi for nomi in nomi_res])) / (len(mi_res) + len(nomi_res))
    if show_results:
        print('MI results: ', mi_print, 'FNR: {:.3f}'.format(1-mi_acc))
        print('NOMI results: ', nomi_print, 'FPR: {:.3f}'.format(1-nomi_acc))
        print('Accuracy: {:.3f}'.format(acc))
    return acc

In [203]:
split = 0.6
train_mi, train_nomi, test_mi, test_nomi = get_train_test(right, left, split=split)
acc = get_results(train_mi, train_nomi, test_mi, test_nomi)

MI results:  ['MI', 'NOMI', 'MI', 'MI'] FNR: 0.250
NOMI results:  ['NOMI', 'MI', 'MI', 'MI'] FPR: 0.750
Accuracy: 0.500


In [197]:
def get_average_results(mi, nomi, split, num_tests):
    accs = []
    for i in range(num_tests):
        train_mi, train_nomi, test_mi, test_nomi = get_train_test(mi, nomi, split)
        accs.append(get_results(train_mi, train_nomi, test_mi, test_nomi, False))
    print('Average accuracy: {:.3f}'.format(np.mean(accs)))

In [209]:
get_average_results(right, nomi, split=0.8, num_tests=100)

Average accuracy: 0.537


In [54]:
numFilt = 8
filers, clf = CCACSP.train(mi, nomi, numFilt)

In [120]:
mi_test, nomi_test = get_all_data(1, bci2_files[1])

DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MILF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MIRF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MISw.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MITy.txt


In [121]:
nomi_test = nomi_test[:12]

In [122]:
mi_test.shape, nomi_test.shape

((12, 8, 1250), (12, 8, 1250))

In [123]:
mi_res = []
nomi_res = []
for i in range(mi_test.shape[0]):
    mi_result = CCACSP.test(mi_test[i], filers, clf)
    mi_res.append(mi_result)
    print("True class: MI, Predicted: {}".format(mi_result[0]))
    nomi_result = CCACSP.test(nomi_test[i], filers, clf)
    nomi_res.append(nomi_result)
    print("True class: NOMI, Predicted: {}".format(nomi_result[0]))

True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 1.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 1.0
True class: NOMI, Predicted: 1.0


In [124]:
np.mean(mi_res)

0.16666666666666666

In [125]:
np.mean(nomi_res)

1.0